In [1]:
import numpy as np
import pandas as pd
#匯入Yahoo!Finance
#pip install 
import yfinance as yf
import datetime as dt

In [2]:
yf.pdr_override()

#可以替換成其他貨幣
# https://finance.yahoo.com/quote/BTC-USD?p=BTC-USD&.tsrc=fin-srch

symbol = "BTC-USD"
start =dt.date(2018, 1, 1)
end = dt.date(2022, 12, 31)

# Read data
df = yf.download(symbol)
df_Indicators=yf.download(symbol)
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2023-07-08,30346.921875,30374.437500,30080.160156,30292.541016,30292.541016,7509378699
2023-07-09,30291.611328,30427.589844,30085.591797,30171.234375,30171.234375,7903327692
2023-07-10,30172.423828,31026.083984,29985.394531,30414.470703,30414.470703,14828209155


In [3]:
def SMA(close,n):   
    return close.rolling(window=n).mean()

In [4]:
def WMA(close, n):
    weights = np.array(range(1, n+1))
    sum_weights = np.sum(weights)

    res = close.rolling(window=n).apply(lambda x: np.sum(weights*x) / sum_weights, raw=False)
    return res

In [5]:
def EMA(close, n):    
    res=close.ewm(span=n,min_periods=n).mean()
    return res

In [6]:
def RSI(close, n):
    # RSI
    change = close.diff(1)
    df["Gain"] = change.mask(change < 0, 0)
    df["Loss"] = abs(change.mask(change > 0, 0))
    df["AVG_Gain"] = df.Gain.rolling(n).mean()
    df["AVG_Loss"] = df.Loss.rolling(n).mean()
    df["RS"] = df["AVG_Gain"] / df["AVG_Loss"]
    df["RSI"] = 100 - (100 / (1 + df["RS"]))
    return df["RSI"]


In [7]:
def Stoch_RSI(close, n):
    #Stochastic RSI
    RSI=df["RSI"]
    LL_RSI = RSI.rolling(n).min()
    HH_RSI = RSI.rolling(n).max()         
    return (RSI - LL_RSI) / (HH_RSI - LL_RSI)

In [8]:
def BB(close, n):
    MA = pd.Series(close.rolling(n).mean())
    STD = pd.Series(close.rolling(n).std())
    bb1 = MA + 2 * STD
    df["Upper Bollinger Band"] = pd.Series(bb1)
    bb2 = MA - 2 * STD
    df["Lower Bollinger Band"] = pd.Series(bb2)
    return df["Upper Bollinger Band"],df["Lower Bollinger Band"]

In [9]:
def MACD(close):
    df['EMA12'] = close.ewm(span=12, adjust=False).mean() 
    df['EMA26'] = close.ewm(span=26, adjust= False).mean() 
    return df['EMA12'] - df['EMA26'] 


In [10]:
# 指標副程式(資料,天數)
df_Indicators['SMA_7']=SMA(df['Adj Close'],7)
df_Indicators['SMA_14']=SMA(df['Adj Close'],14)

df_Indicators['EMA_7']=EMA(df['Adj Close'],7)
df_Indicators['EMA_14']=EMA(df['Adj Close'],14)

df_Indicators['RSI']=RSI(df['Adj Close'],14)


df_Indicators['Stoch_RSI']=Stoch_RSI(df['Adj Close'],14)

df_Indicators['Upper_BB']=BB(df['Adj Close'],20)[0]
df_Indicators['Lower_BB']=BB(df['Adj Close'],20)[1]

df_Indicators['MACD']=MACD(df['Adj Close'])
# df_Indicators= df_Indicators.dropna()
df_Indicators

,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD
Date,,,,,,,,,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.624024
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-7.014744
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-9.249402
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-11.699137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-08,30346.921875,30374.437500,30080.160156,30292.541016,30292.541016,7509378699,30516.157366,30475.113281,30376.692554,30147.162349,46.873470,0.029407,32065.424966,28235.959605,744.169383
2023-07-09,30291.611328,30427.589844,30085.591797,30171.234375,30171.234375,7903327692,30451.938058,30453.039900,30325.328009,30150.371952,46.276183,0.013215,31438.347378,29195.057701,683.094027
2023-07-10,30172.423828,31026.083984,29985.394531,30414.470703,30414.470703,14828209155,30345.942522,30463.278460,30347.613683,30185.585119,51.713181,0.160609,31038.469782,29803.633538,646.861885


In [11]:
df_Indicators.to_csv('BTC.csv')

In [43]:
import pandas as pd
from glob import glob
 
files =['BTC-Indicators.csv','data\CionScore_final.csv']
 
df = pd.concat((pd.read_csv(file) for file in files), axis='columns').drop(['Unnamed: 0','CoinDate'],axis=1)
df= df.dropna()

df.to_csv('data/BTC_final.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,SMA_7,SMA_14,EMA_7,EMA_14,RSI,Stoch_RSI,Upper_BB,Lower_BB,MACD,ScorePositive,ScoreNegative,ScoreNeutral
27,2018-01-28,11475.299805,12040.299805,11475.299805,11786.299805,11786.299805,8.350360e+09,11259.571708,11635.535854,11516.047002,11884.539097,37.129744,0.560353,15192.188206,9606.291969,-761.452864,0.023502,0.057272,0.919226
28,2018-01-29,11755.500000,11875.599609,11179.200195,11296.400391,11296.400391,7.107360e+09,11311.714565,11455.293039,11461.122271,11804.864485,34.531168,0.835160,14866.144405,9602.435771,-744.090581,0.163158,0.294283,0.542558
29,2018-01-30,11306.799805,11307.200195,10036.200195,10106.299805,10106.299805,8.637860e+09,11202.843052,11356.421596,11122.356157,11575.251876,40.137091,1.000000,14450.867575,9531.012601,-816.944788,0.102038,0.260553,0.637410
30,2018-01-31,10108.200195,10381.599609,9777.419922,10221.099609,10221.099609,8.041160e+09,11040.228655,11287.314453,10897.011838,11392.534553,42.917380,1.000000,14318.165125,9345.245031,-855.556563,0.098884,0.268044,0.633072
31,2018-02-01,10237.299805,10288.799805,8812.280273,9170.540039,9170.540039,9.959400e+09,10741.820033,11122.717285,10465.350527,11093.196564,34.828472,0.651247,14132.400116,9050.004084,-959.863343,0.075772,0.452595,0.471633
